In [1]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import sqlalchemy as sa
from multiprocessing import Pool
%matplotlib inline

In [2]:
!ls -lh currents

total 3.4G
-rw-rw-r-- 1 ray ray 6.3M Aug 14 01:58 AAL.pkl
-rw-rw-r-- 1 ray ray 4.2M Aug 14 04:00 AAPL.pkl
-rw-rw-r-- 1 ray ray 3.4M Aug 14 02:06 AAP.pkl
-rw-rw-r-- 1 ray ray 3.0M Aug 14 01:48 ABBV.pkl
-rw-rw-r-- 1 ray ray 3.4M Aug 14 04:30 ABC.pkl
-rw-rw-r-- 1 ray ray 2.0M Aug 14 01:54 ABMD.pkl
-rw-rw-r-- 1 ray ray 4.4M Aug 14 01:47 ABT.pkl
-rw-rw-r-- 1 ray ray 3.4M Aug 14 01:56 ACN.pkl
-rw-rw-r-- 1 ray ray 6.4M Aug 14 02:01 ADBE.pkl
-rw-rw-r-- 1 ray ray 4.4M Aug 14 03:46 ADI.pkl
-rw-rw-r-- 1 ray ray 6.1M Aug 14 01:48 ADM.pkl
-rw-rw-r-- 1 ray ray 4.3M Aug 14 02:07 ADP.pkl
-rw-rw-r-- 1 ray ray 4.8M Aug 14 02:04 ADSK.pkl
-rw-rw-r-- 1 ray ray 5.2M Aug 14 04:14 ADS.pkl
-rw-rw-r-- 1 ray ray  13M Aug 14 01:55 AEE.pkl
-rw-rw-r-- 1 ray ray  37M Aug 14 02:06 AEP.pkl
-rw-rw-r-- 1 ray ray  12M Aug 14 02:09 AES.pkl
-rw-rw-r-- 1 ray ray 9.1M Aug 14 02:12 AET.pkl
-rw-rw-r-- 1 ray ray  18M Aug 14 03:52 AFL.pkl
-rw-rw-r-- 1 ray ray 2.6M Aug 14 04:12 AGN.pkl
-rw-rw-r-- 1 ray ray  26M Aug 14 02:13 AIG.p

In [3]:
!du -h currents

4.0K	currents/.ipynb_checkpoints
3.4G	currents


In [4]:
tickers_in_currents = !ls currents
tickers_in_currents = [t.replace('.pkl','') for t in tickers_in_currents]

In [5]:
len(tickers_in_currents)

464

In [6]:
df_sp500 = pd.read_pickle('sp500.pkl')

In [21]:
tickers_not_included = set(df_sp500['Ticker symbol']) - set(tickers_in_currents)

In [22]:
tickers_not_included

{'ANDV',
 'ANTM',
 'AON',
 'APTV',
 'ARNC',
 'BF.B',
 'BHF',
 'BHGE',
 'BKNG',
 'BRK.B',
 'CBRE',
 'CFG',
 'DISCK',
 'DWDP',
 'DXC',
 'EVRG',
 'FOX',
 'FOXA',
 'FTV',
 'GOOGL',
 'HPE',
 'INFO',
 'IQV',
 'KHC',
 'LB',
 'M',
 'MNST',
 'NWSA',
 'PYPL',
 'QRVO',
 'SEE',
 'SPGI',
 'SYF',
 'TPR',
 'UA',
 'UAA',
 'VIAB',
 'WBA',
 'WLTW',
 'WRK',
 'ZBH'}

In [9]:
len(tickers_not_included)

41

In [11]:
df_t = pd.read_pickle('currents/DISH.pkl')

In [2]:
con = sa.create_engine('sqlite:///edgar_htm_idx.db').connect()
ticker_df = pd.read_sql('select date, tc.Ticker from idx inner join tickercik tc on tc.CIK=idx.cik where tc.Ticker in {} and type in ("10-K","10-Q")'.format(tuple(tickers_in_currents)), con)

In [7]:
ticker_df

,date,Ticker
0,2009-02-17,MMM
1,2009-02-20,ABT
2,2009-02-09,ABMD
3,2009-03-27,ACN
4,2009-01-23,ADBE
5,2009-03-04,AAP
6,2009-02-24,AMD
7,2009-02-26,AES
8,2009-02-27,AET
9,2009-03-02,AMG


In [12]:
df_t

,Value,Account,Category,Period,ContextID,isStartEndPeriod,isInstantPeriod,isForeverPeriod,startDateTime,endDateTime
0,208369696,dei:EntityCommonStockSharesOutstanding,True,2009-07-24,BalanceAsOf_24Jul2009_Class_A_common_stock,False,True,False,NaT,2009-07-25
1,238435208,dei:EntityCommonStockSharesOutstanding,True,2009-07-24,BalanceAsOf_24Jul2009_Class_B_common_stock,False,True,False,NaT,2009-07-25
2,5900000000,dei:EntityPublicFloat,False,2008-06-30,BalanceAsOf_30Jun2008,False,True,False,NaT,2008-07-01
3,1020086000,dish:AccruedProgramming,False,2008-12-31,BalanceAsOf_31Dec2008,False,True,False,NaT,2009-01-01
4,961416000,dish:AccruedProgramming,False,2009-06-30,BalanceAsOf_30Jun2009,False,True,False,NaT,2009-07-01
5,72086000,dish:ChangeInLongTermDeferredRevenueDistributi...,False,2008-01-012008-06-30,SixMonthsEnded_30Jun2008,True,False,False,2008-01-01,2008-07-01
6,47606000,dish:ChangeInLongTermDeferredRevenueDistributi...,False,2009-01-012009-06-30,January-01-2009_June-30-2009,True,False,False,2009-01-01,2009-07-01
7,-105000,dish:ChangeInRestrictedCashAndMarketableInvest...,False,2008-01-012008-06-30,SixMonthsEnded_30Jun2008,True,False,False,2008-01-01,2008-07-01
8,-62473000,dish:ChangeInRestrictedCashAndMarketableInvest...,False,2009-01-012009-06-30,January-01-2009_June-30-2009,True,False,False,2009-01-01,2009-07-01
9,711871000,dish:DepositToAcquireSpectrum,False,2008-01-012008-06-30,SixMonthsEnded_30Jun2008,True,False,False,2008-01-01,2008-07-01


In [35]:
%%time

def get_df(f):
    try:
        df_t = pd.read_pickle(f)
        df_t = df_t[df_t.Category==False]
        df_t['ticker'] = f.replace('currents/','').replace('.pkl','')
        return df_t
    except Exception as e:
        print(e)
        return

p = Pool(5)
dfs = p.map(get_df, glob.glob('currents/*.pkl'))


CPU times: user 51 s, sys: 8.74 s, total: 59.8 s
Wall time: 59.5 s


In [38]:
df = pd.concat(dfs, ignore_index=True)

In [39]:
df.shape

(7491869, 11)

In [42]:
df.to_pickle('sp500_all.pkl')